In [1]:
import pandas as pd
import numpy as np
import os
import glob

# 系统参数
r = 0.05  # 系统的年化率 
n = 20    # 设备的使用年限

# 年化因子计算
annualization_factor = (r * (1 + r)**n) / ((1 + r)**n - 1)

# 2020年基准成本参数（来自disaster_2020.py）
investment_cost_2020 = {
    'WT': 1392, 'PV': 1377, 'WEC': 6000, 'AC': 150, 'EB': 250, 'CHP': 1300,
    'PEM': 1371, 'FC': 3000, 'LNG': 700, 'LNGV': 500, 'ESS': 1365, 
    'TES': 250, 'CES': 250, 'H2S': 50
}

fixed_om_cost_2020 = {
    'WT': 43, 'PV': 23, 'LNG': 14, 'WEC': 300, 'CHP': 26, 'EB': 5, 'AC': 3, 
    'PEM': 41, 'FC': 90, 'LNGV': 25, 'ESS': 34, 'TES': 6, 'CES': 6, 'H2S': 1.2
}


In [2]:
def calculate_island_costs_corrected(island_capacity_file, output_2020_dir, population_file):
    """
    使用正确的年化投资成本和固定运维成本公式计算岛屿能源系统的六项成本
    并计算人均成本
    """
    # 读取容量配置数据
    capacity_df = pd.read_csv(island_capacity_file)
    
    # 读取人口数据
    pop_df = pd.read_csv(population_file)
    print(f"人口数据文件包含 {len(pop_df)} 行数据")
    
    # 初始化结果列表
    results = []
    
    for idx, row in capacity_df.iterrows():
        lat, lon = row['lat'], row['lon']
        
        # 构建对应的best_cost文件路径
        cost_file = os.path.join(output_2020_dir, f"{lat}_{lon}_best_cost.csv")
        
        if not os.path.exists(cost_file):
            print(f"警告: 文件不存在 {cost_file}")
            continue
        
        # 查找对应的人口数据
        # 使用经纬度匹配（允许小的误差）
        pop_match = pop_df[(abs(pop_df['Lat'] - lat) < 0.01) & (abs(pop_df['Long'] - lon) < 0.01)]
        
        if len(pop_match) == 0:
            print(f"警告: 未找到岛屿 ({lat}, {lon}) 的人口数据")
            continue
        elif len(pop_match) > 1:
            print(f"警告: 岛屿 ({lat}, {lon}) 找到多个人口数据，使用第一个")
            
        # 获取人口数据，限制最大值为500
        population = min(pop_match.iloc[0]['pop'], 500)

        # 读取成本数据
        cost_df = pd.read_csv(cost_file)
        cost_dict = dict(zip(cost_df['Cost_Item'], cost_df['Cost_Value']))
        
        # 使用正确的成本计算公式
        
        # 1. 计算可再生能源设施的年化投资成本和固定运维成本
        renewable_devices = ['PV', 'WT', 'WEC']
        renewable_annualized_investment = sum(
            row[device] * investment_cost_2020[device] * annualization_factor 
            for device in renewable_devices if device in row.index
        )
        renewable_fixed_om = sum(
            row[device] * fixed_om_cost_2020[device] 
            for device in renewable_devices if device in row.index
        )
        renewable_cost = renewable_annualized_investment + renewable_fixed_om
        
        # 2. 计算储能设施的年化投资成本和固定运维成本
        storage_devices = ['ESS', 'TES', 'CES', 'H2S']
        storage_annualized_investment = sum(
            row[device] * investment_cost_2020[device] * annualization_factor 
            for device in storage_devices if device in row.index
        )
        storage_fixed_om = sum(
            row[device] * fixed_om_cost_2020[device] 
            for device in storage_devices if device in row.index
        )
        storage_cost = storage_annualized_investment + storage_fixed_om
        
        # 3. 计算其他能源转换设施的年化投资成本和固定运维成本
        other_devices = ['CHP', 'EB', 'AC', 'PEM', 'FC', 'LNGV']
        other_annualized_investment = sum(
            row[device] * investment_cost_2020[device] * annualization_factor 
            for device in other_devices if device in row.index
        )
        other_fixed_om = sum(
            row[device] * fixed_om_cost_2020[device] 
            for device in other_devices if device in row.index
        )
        other_equipment_cost = other_annualized_investment + other_fixed_om
        
        # 4. 计算LNG总成本 (年化投资成本 + 固定运维成本 + 购买成本)
        lng_annualized_investment = row['LNG'] * investment_cost_2020['LNG'] * annualization_factor
        lng_fixed_om = row['LNG'] * fixed_om_cost_2020['LNG']
        lng_purchase_cost = cost_dict.get('LNG Purchase Cost', 0)
        lng_cost = lng_annualized_investment + lng_fixed_om + lng_purchase_cost
        
        # 5. 获取discard cost (能源丢弃成本)
        discard_cost = (cost_dict.get('Energy Discard Cost (Heat/Cold)', 0) + 
                       cost_dict.get('Renewable Curtailment Cost', 0))
        
        # 6. 获取load shedding cost (负荷削减成本)
        load_shedding_cost = cost_dict.get('Load Shedding Cost', 0)
        
        # 计算人均成本
        renewable_cost_per_capita = renewable_cost / population
        storage_cost_per_capita = storage_cost / population
        lng_cost_per_capita = lng_cost / population
        other_equipment_cost_per_capita = other_equipment_cost / population
        discard_cost_per_capita = discard_cost / population
        load_shedding_cost_per_capita = load_shedding_cost / population
        
        # 保存结果
        results.append({
            'lat': lat,
            'lon': lon,
            'population': population,
            'original_population': pop_match.iloc[0]['pop'],
            # 总成本
            'renewable_cost': renewable_cost,
            'storage_cost': storage_cost,
            'lng_cost': lng_cost,
            'other_equipment_cost': other_equipment_cost,
            'discard_cost': discard_cost,
            'load_shedding_cost': load_shedding_cost,
            # 人均成本
            'renewable_cost_per_capita': renewable_cost_per_capita,
            'storage_cost_per_capita': storage_cost_per_capita,
            'lng_cost_per_capita': lng_cost_per_capita,
            'other_equipment_cost_per_capita': other_equipment_cost_per_capita,
            'discard_cost_per_capita': discard_cost_per_capita,
            'load_shedding_cost_per_capita': load_shedding_cost_per_capita,
            # 额外详细信息
            'renewable_investment': renewable_annualized_investment,
            'renewable_om': renewable_fixed_om,
            'storage_investment': storage_annualized_investment,
            'storage_om': storage_fixed_om,
            'other_investment': other_annualized_investment,
            'other_om': other_fixed_om,
            'lng_investment': lng_annualized_investment,
            'lng_om': lng_fixed_om,
            'lng_purchase': lng_purchase_cost
        })
    
    return pd.DataFrame(results)

# 执行修正后的计算（加入人口数据）
island_capacity_file = '../result/island_capacity_2020.csv'
output_2020_dir = '../result/output_2020'
population_file = 'filtered_island_1898.csv'

cost_summary_df = calculate_island_costs_corrected(island_capacity_file, output_2020_dir, population_file)
print(f"成功处理 {len(cost_summary_df)} 个岛屿的成本数据")
print("\n前5行预览（人均成本）：")
per_capita_cols = ['lat', 'lon', 'population', 'renewable_cost_per_capita', 'storage_cost_per_capita', 'lng_cost_per_capita', 'other_equipment_cost_per_capita']
print(cost_summary_df[per_capita_cols].head())

人口数据文件包含 1899 行数据
警告: 岛屿 (11.103808, -16.127637) 找到多个人口数据，使用第一个
成功处理 1898 个岛屿的成本数据

前5行预览（人均成本）：
        lat         lon  population  renewable_cost_per_capita  \
0 -0.030217  103.626017         235                 386.060691   
1 -0.035119  103.659208          59                 430.186591   
2 -0.045282  130.124424         218                 421.867363   
3 -0.061393   98.276855         500                 429.209762   
4 -0.074667   98.326246         500                 429.440715   

   storage_cost_per_capita  lng_cost_per_capita  \
0               106.104298                  0.0   
1                66.456088                  0.0   
2                82.180846                  0.0   
3                83.964073                  0.0   
4                84.125223                  0.0   

   other_equipment_cost_per_capita  
0                        55.196700  
1                         8.754898  
2                         9.009407  
3                         9.943355  
4             

In [3]:
# 添加IPCC区域信息
import geopandas as gpd
from shapely.geometry import Point

def add_ipcc_regions(df):
    """
    根据IPCC geojson数据为每个岛屿添加区域信息
    """
    # 读取IPCC区域数据
    try:
        ipcc_regions = gpd.read_file('IPCC-WGI-reference-regions-v4.geojson')
        print(f"成功读取IPCC区域数据，共{len(ipcc_regions)}个区域")
        
        # 为每个岛屿创建Point几何
        geometry = [Point(lon, lat) for lon, lat in zip(df['lon'], df['lat'])]
        islands_gdf = gpd.GeoDataFrame(df, geometry=geometry, crs='EPSG:4326')
        
        # 空间连接，找到每个岛屿对应的IPCC区域
        islands_with_regions = gpd.sjoin(islands_gdf, ipcc_regions[['Name', 'Acronym', 'Continent', 'geometry']], 
                                       how='left', predicate='within')
        
        # 清理结果，保留原有列和新增的区域信息
        result_df = islands_with_regions.drop(['geometry', 'index_right'], axis=1, errors='ignore')
        
        # 重命名列
        if 'Name' in result_df.columns:
            result_df = result_df.rename(columns={'Name': 'IPCC_Region_Name', 'Acronym': 'IPCC_Region_Code'})
        
        # 处理未匹配的岛屿
        unmatched = result_df[result_df['IPCC_Region_Name'].isna()]
        if len(unmatched) > 0:
            print(f"警告: {len(unmatched)}个岛屿未能匹配到IPCC区域:")
            for idx, row in unmatched.iterrows():
                print(f"  岛屿 ({row['lat']:.3f}, {row['lon']:.3f})")
        
        print(f"成功匹配 {len(result_df) - len(unmatched)} 个岛屿到IPCC区域")
        return result_df
        
    except Exception as e:
        print(f"读取IPCC区域数据失败: {e}")


# 为成本数据添加区域信息
cost_summary_with_regions = add_ipcc_regions(cost_summary_df.copy())

# 显示区域统计
print("\n=== 区域分布统计 ===")
region_stats = cost_summary_with_regions.groupby(['IPCC_Region_Name', 'IPCC_Region_Code']).size().reset_index(name='Island_Count')
print(region_stats.sort_values('Island_Count', ascending=False))

# 显示前几行包含区域信息的数据
print(f"\n=== 带区域信息的成本数据框预览 ===")
display_cols = ['lat', 'lon', 'IPCC_Region_Name', 'IPCC_Region_Code', 'renewable_cost', 'storage_cost', 'lng_cost', 'other_equipment_cost']
print(cost_summary_with_regions[display_cols].head(10))

成功读取IPCC区域数据，共58个区域
成功匹配 1898 个岛屿到IPCC区域

=== 区域分布统计 ===
             IPCC_Region_Name IPCC_Region_Code  Island_Count
33                   S.E.Asia              SEA           703
6                      E.Asia              EAS           175
22                   N.Europe              NEU           162
15              Mediterranean              MED           115
29                     S.Asia              SAS            88
12   Equatorial.Pacific-Ocean              EPO            88
37            S.Pacific-Ocean              SPO            86
4                   Caribbean              CAR            71
8             E.North-America              ENA            45
45             Western-Africa              WAF            31
17                N.Australia              NAU            30
16           N.Atlantic-Ocean              NAO            26
39          S.W.South-America              SWS            26
25          N.W.North-America              NWN            25
42            W.North-Americ

In [4]:
# 保存结果到CSV文件
output_file = 'island_cost_summary_2020.csv'
cost_summary_with_regions.to_csv(output_file, index=False)
print(f"\n成本汇总数据已保存到: {output_file}")


成本汇总数据已保存到: island_cost_summary_2020.csv
